In [1]:
var SparkContext = require("eclairjs/SparkContext")
var SQLContext = require("eclairjs/sql/SQLContext")

In [2]:
var sc = new SparkContext("local[*]", "foo");
var sqlContext = new SQLContext(sc);

In [3]:
var rdd = sc.textFile(
  "/Users/bburns/dev/demandtec/data/streaming/*"
);

In [4]:
var DataTypes = require("eclairjs/sql/types/DataTypes");

var fields = [];
fields.push(DataTypes.createStructField("product_id", DataTypes.StringType, true));
fields.push(DataTypes.createStructField("store_id", DataTypes.StringType, true));
fields.push(DataTypes.createStructField("date", DataTypes.DateType, true));
fields.push(DataTypes.createStructField("time", DataTypes.TimestampType, true));
fields.push(DataTypes.createStructField("units", DataTypes.DoubleType, true));
fields.push(DataTypes.createStructField("price", DataTypes.DoubleType, true));
fields.push(DataTypes.createStructField("type", DataTypes.StringType, true));

var schema = DataTypes.createStructType(fields);

7

In [5]:
var trainingRDD =  rdd.map(function(line, Tuple, SqlDate, SqlTimestamp) {
    var v = line.split("|")
        //product, location, date, time, units sold, price
    return new Tuple(v[7], v[2], v[4], v[5], parseFloat(v[8]), parseFloat(v[10]), v[6]);
}, [require("eclairjs/Tuple")]).filter(function(x) {
        //filter out 0 units sold and 0 price
    return (x[5] > 0 && x[4] > 0 && x[6] == "SALE");
}).map(function(t, RowFactory, SqlDate, SqlTimestamp) {
    var parts = t[2].match(/(\d+)/g);
    var date = new Date(parts[0], parts[1]-1, parts[2]);
    var time = new Date(Date.parse(t[2]+" "+t[3]));
    var row = new RowFactory.create([
        t[0], 
        t[1], 
        new SqlDate(date),
        new SqlTimestamp(time), 
        parseFloat(t[4]), 
        parseFloat(t[5]), 
        t[6]]);
    return row;
}, [require("eclairjs/sql/RowFactory"),require("eclairjs/sql/SqlDate"),require("eclairjs/sql/SqlTimestamp")]);

var df = sqlContext.createDataFrame(trainingRDD, schema);

In [6]:
var df2 = sqlContext.read().parquet("/Users/bburns/dev/demandtec/data/models/dates/V11_arr")

In [7]:
var df3 = df.join(df2, "date")

In [8]:
var rows = df3.take(10)

In [9]:
var df4 = sqlContext.createDataFrame(rows, df3.schema())

In [10]:
df4.show()

In [11]:
df3.printSchema()